CREAR Ciclos Poligonales Equivalentes

In [1]:
import pandas as pd
import numpy as np
import os
import plotly.graph_objects as go
# Función para calculo de aceleracion. h ~ datos por segundo
def calcular_acel(df,h): # en m/s2
    vel = df["v_km_h_f"]
    v = np.array(vel)/3.6  # Pasa de km/h a m/s
    n = len(v)
    a = np.zeros(n)
    for i in range(n):
        if i == 0:
            a[i]=(-v[i+2] + 4*v[i+1] - 3*v[i])/(2*h)
            #print("i=0")
        elif i == 1:
            a[i]=(v[i+1] - v[i-1])/(2*h)
            #print("i=1")
        elif i == n-2:
            a[i]=(v[i+1] - v[i-1])/(2*h)
            #print("i=n-2")
        elif i == n-1:
            a[i]=(3*v[i] - 4*v[i-1] + v[i-2])/(2*h)
            #print("i=n-1")
        else:
            a[i] = (-v[i+2] + 8*v[i+1] - 8*v[i-1] + v[i-2])/(12*h)
    return(a)

import pandas as pd



def calcular_parametros_acel(df):
    Data_time = 1 # Un dato por segundo
    df = df.reset_index(drop=True) # Reindexar el DF
    df["v_m_s_f"] = df["v_km_h_f"]/3.6
    df["t_s"] = range(len(df)) # Crear nueva columna de tiempo con una frecuencia de 1hz
    df["acel_m_s2"] = calcular_acel(df,Data_time) 
    df['ABS_Aceleracion'] = abs(df['acel_m_s2'])
    df['Acel_cumul'] = df['ABS_Aceleracion'].cumsum()
    df['Derivada_Aceleracion'] = df['acel_m_s2'].diff()
    df['Derivada_Aceleracion'].fillna(0, inplace=True)
    df['ABS_Derivada_Aceleracion'] = abs(df['Derivada_Aceleracion'])
    df['Der_Acel_cumul'] = df['ABS_Derivada_Aceleracion'].cumsum()
    return df

In [12]:
import pandas as pd
import numpy as np


import pandas as pd
import numpy as np

def velocidad_poligonal(df):
    # Definir la tolerancia para la suavidad de la curva
    tolerancia = 0.1  # Unidad: m/s

    # Crear una lista para almacenar los vértices de la curva poligonal
    vertices = []

    # Iterar sobre el DataFrame para generar la curva poligonal
    for i in range(len(df) - 1):
        diff_velocidad = abs(df['v_m_s_f'][i + 1] - df['v_m_s_f'][i])
        if diff_velocidad > tolerancia:
            vertices.append((df['t_s'][i], df['v_m_s_f'][i]))

    # Añadir el último punto del DataFrame como el último vértice
    vertices.append((df['t_s'].iloc[-1], df['v_m_s_f'].iloc[-1]))

    # Convertir la lista de vértices a un DataFrame para facilitar su manipulación
    df_vertices = pd.DataFrame(vertices, columns=['t_s', 'v_m_s_f'])

    # Crear una función para interpolar la curva poligonal
    def interpolar_velocidad(tiempo):
        velocidad = np.interp(tiempo, df_vertices['t_s'], df_vertices['v_m_s_f'])
        return velocidad

    # Agregar la curva poligonal como una nueva columna al DataFrame original
    df['vel_poligonal'] = interpolar_velocidad(df['t_s'])
    df['vel_poligonal'][df['vel_poligonal'] <= 1] = 0

    return df

def velocidad_poligonal2(df):
    # Definir la tolerancia para la suavidad de la curva
    tolerancia = 0.05  # Unidad: m/s

    # Crear una lista para almacenar los vértices de la curva poligonal
    vertices = []

    # Iterar sobre el DataFrame para generar la curva poligonal
    for i in range(len(df) - 1):
        diff_velocidad = abs(df['vel_poligonal'][i + 1] - df['vel_poligonal'][i])
        if diff_velocidad > tolerancia:
            vertices.append((df['t_s'][i], df['vel_poligonal'][i]))

    # Añadir el último punto del DataFrame como el último vértice
    vertices.append((df['t_s'].iloc[-1], df['vel_poligonal'].iloc[-1]))

    # Convertir la lista de vértices a un DataFrame para facilitar su manipulación
    df_vertices = pd.DataFrame(vertices, columns=['t_s', 'vel_poligonal'])

    # Crear una función para interpolar la curva poligonal
    def interpolar_velocidad(tiempo):
        velocidad = np.interp(tiempo, df_vertices['t_s'], df_vertices['vel_poligonal'])
        return velocidad

    # Agregar la curva poligonal como una nueva columna al DataFrame original
    df['vel_poligonal'] = interpolar_velocidad(df['t_s'])
    df['vel_poligonal'][df['vel_poligonal'] <= 1] = 0

    return df



In [13]:
# Ruta del directorio donde se encuentran los archivos Excel originales
directorio_origen = 'C:/Users/Lenovo/OneDrive - Universidad de Castilla-La Mancha/JAVIER_LONDONO/RDE_Development/02.Road_Altitude'

# Ruta del directorio donde se guardarán los archivos Excel modificados
directorio_destino = 'C:/Users/Lenovo/OneDrive - Universidad de Castilla-La Mancha/JAVIER_LONDONO/RDE_Development/03.Clasificacion/Intervalos_poligonales'

# Lista para almacenar los nombres de los archivos Excel en el directorio de origen
archivos_excel = []

# Recorrer el directorio de origen y guardar los nombres de los archivos Excel
for archivo in os.listdir(directorio_origen):
    if archivo.endswith('.xlsx'):
        archivos_excel.append(archivo)

# Iterar sobre cada archivo Excel
for archivo in archivos_excel:
    # Ruta completa del archivo de origen
    ruta_origen = os.path.join(directorio_origen, archivo)
    
    # Cargar el archivo Excel en un DataFrame
    df = pd.read_excel(ruta_origen)
    
    # Aplicar la función para calcular los parámetros de aceleración
    df_modificado = calcular_parametros_acel(df)
    
    # Aplicar la función para calcular velocidad poligonal
    df_modificado_poligonal = velocidad_poligonal(df_modificado)
    df_modificado_poligonal = velocidad_poligonal2(df_modificado)    
    # Generar la ruta de destino para guardar el archivo modificado
    ruta_destino = os.path.join(directorio_destino, archivo)
    
    # Guardar el DataFrame modificado como un nuevo archivo Excel
    df_modificado_poligonal.to_excel(ruta_destino, index=False)

C:\Users\Lenovo\AppData\Local\Temp\ipykernel_7120\2621381374.py:34: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\Lenovo\AppData\Local\Temp\ipykernel_7120\2621381374.py:64: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\Lenovo\AppData\Local\Temp\ipykernel_7120\2621381374.py:34: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\Lenovo\AppData\Local\Temp\ipykernel_7120\2621381374.py:64: SettingWithCopyWarning:


A v

In [14]:
origen_poligonal = 'C:/Users/Lenovo/OneDrive - Universidad de Castilla-La Mancha/JAVIER_LONDONO/RDE_Development/03.Clasificacion/Intervalos_poligonales/'

Urban_1 = pd.read_excel(origen_poligonal + "Interval_Urban_02042024.xlsx")
Urban_2 = pd.read_excel(origen_poligonal + "Interval_Urban_15042024.xlsx")
Urban_3 = pd.read_excel(origen_poligonal + "Interval_Urban_19042024.xlsx")

Rural_1 = pd.read_excel(origen_poligonal + "Interval_Rural_05042024.xlsx")
Rural_2 = pd.read_excel(origen_poligonal + "Interval_Rural_15042024.xlsx")
Rural_3 = pd.read_excel(origen_poligonal + "Interval_Rural_29042024.xlsx")

Motorway_1 = pd.read_excel(origen_poligonal + "Interval_Motorway_05042024.xlsx")
Motorway_2 = pd.read_excel(origen_poligonal + "Interval_Motorway_15042024.xlsx")
Motorway_3 = pd.read_excel(origen_poligonal + "Interval_Motorway_29042024.xlsx")

In [15]:
# Velocidad



fig = go.Figure(data=[
    go.Scatter(x=Urban_1["t_s"], y=Urban_1["v_m_s_f"], name="v_m_s_f", mode="lines", line=dict(color='darkblue')),
    go.Scatter(x=Urban_1["t_s"], y=Urban_1["vel_poligonal"], name="vel_poligonal", mode="lines", line=dict(color='darkorange'))])

# Actualizar ejes
fig.update_xaxes(zeroline=True, zerolinewidth=1, zerolinecolor='black', title_text="Time [s]")
fig.update_yaxes(zeroline=True, zerolinewidth=1, zerolinecolor='black', title_text="Speed ")

# Actualizar leyenda
fig.update_layout(legend=dict(orientation="h", x=0.3, y=1.3), height=500, width=900, plot_bgcolor="white")
fig.show()